In [1]:
import ee

In [2]:
ee.Initialize()

### Define premise parameter

In [3]:
# define the year range
year_start = '2017-01-01'
year_end   = '2019-12-31'

year_name = f'{year_start[:4]}_{year_end[:4]}'

# import the research area boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain/0_1_North_China_Plain_Full")

In [4]:
# use if condition to find out the right Landsat img formulation
# here use Landsat Raw data

end_year = int(year_name.split('_')[1])

if  end_year  <= 2010:
    Landsat_img = ee.ImageCollection("LANDSAT/LT05/C01/T1").select(['B4','B3'])

elif end_year <= 2013:
    Landsat_img = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").select(['B4','B3'])

else:
    Landsat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1").select(['B5','B4'])
    

### Filter the landsat imgs and calculate the NDVI

In [5]:
# Calculate the Mean-value NDVI

NDVI = Landsat_img.filterDate(year_start,year_end)\
                  .filterBounds(North_China_Plain)\
                  .mean()\
                  .normalizedDifference()\
                  .clip(North_China_Plain)

In [6]:
# Export the NDVI-img to google storage bucket

# define the name
name = f'Year_{year_name}_Mean_NDVI'

# multiply the NDVI with 100 and change it 
# into a int-img to reduce space
img = NDVI.multiply(100).toInt8()

task = ee.batch.Export.image.toCloudStorage(image           = img,
                                            description     = name,
                                            bucket          = 'fourier_pixels',
                                            fileNamePrefix  = name,
                                            
                                            region          = North_China_Plain.geometry().bounds().getInfo()['coordinates'],
                                            scale           = 30,
                                            
                                            maxPixels       = int(1e13),
                                           
                                            skipEmptyTiles  = True )

task.start()